In [3]:
# Dependencies
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import json
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [7]:
# Target Search Term
target_terms = ("@BBCNews", "@CBSNews", "@CNN",
                "@FoxNews", "@nytimes")
#Final DF for csv
social_analytics_df = pd.DataFrame()

# Loop through all target users
for target in target_terms:

    # Variables for holding sentiments
    news_media = []
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    date_created = []
    tweets_ago = []
    
    # Run search around each tweet
    public_tweets = api.user_timeline(target, count=100)
    # Loop through all tweets
    i = 0
    for tweet in public_tweets:
            
        # Run Vader Analysis on each tweet
        results = analyzer.polarity_scores(tweet["text"])
        tweet_text = tweet["text"]
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]
        date_created.append(datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")) 
        # Add each value to the appropriate array
        compound_list.append(compound)
        positive_list.append(pos)
        negative_list.append(neg)
        neutral_list.append(neu)
        news_media.append(tweet['user']['screen_name'])
        tweets_ago.append(i)
        i=i+1

    #Dictionary
    sentiment_df = pd.DataFrame({
        "News_Media": target,
        "Compound": compound_list,
        "Positive": positive_list,
        "Negative": negative_list,
        "Neutral": neutral_list,
        "Date_created": date_created,
        "Tweets_ago": tweets_ago
#         "Tweet Count": len(compound_list)
    })
    social_analytics_df = social_analytics_df.append(sentiment_df)
    
social_analytics_df.set_index('News_Media', 'Date_Created')
social_analytics_df.to_csv("./news_tweets.csv")

In [15]:
#Create X-axis values
tweets_ago = social_analytics_df.loc[social_analytics_df.News_Media=='@BBCNews']['Tweets_ago']
tweet_compound = social_analytics_df.loc[social_analytics_df.News_Media=='@BBCNews']['Compound']